# Tachometer Notes

In [58]:
# import required packages
import numpy as np
from scipy.fftpack import fft
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
from IPython.display import Image
%matplotlib inline

This is what the IR receiver's signal looks like with 100$\Omega$ on the 3.3V PWM  at 38kHz:
![1kohm](100ohm.bmp "Image from a Rigol DS1102E")

This is what the IR receiver's signal looks like with 1K$\Omega$ on the 3.3V PWM  at 38kHz:
![1kohm](1kohm.bmp "Image from a Rigol DS1102E")

This is what the IR receiver's signal looks like with 1K$\Omega$ on the 3.3V PWM  at 28kHz:
![1kohm](1kohm28khz.bmp "Image from a Rigol DS1102E")

I was having a lot of trouble with the signal jumping all over the place when the reflector wasn't in front of the sensor, so I decided I needed to lower the sensitivity of the receiver. 

The IR receiver is _very_ sensitive to 38kHz IR, but less so as the frequency deviates from that value. I changed the PWM to 28kHz, and now the signal is much cleaner.

Also, I've found it helps to keep the sensor pointed just off from normal to the surface of the reflector so that only the reflector reflects the IR.

With the correct resistor setup, the output voltage is about 2.8V, so this should be enough to trigger the GPIO interrupt.

The problem now is that the signal sits at 3.3V when not triggered, and drops by about 2V when it triggers. This is difficult to get to work with the BBB since the GPIO pin pulls the 3.3V down. Depending on if I put a resistor between the IR receiver and the GPIO pin, the voltage might be at 0.5V or so whether the IR receiver is triggered or not.

Solving the pulldown resistor: use a "Device Tree Overlay" to disable the pulldown resistor on a specific GPIO pin. This will allow for a "high" to be the default state.

Using the main.c script to record voltages on the ain_0 pin works, but doesn't display correctly in excel. Use Notepad++ to view the csv.

Alright, I have a csv with what looks like some good freq data. I need to do something like take each 1 second window and find the freq in that window, then plot it.

So far, David and I have attempted to use a 'rolling window' where we take the FFT and find the peak freq in that window, but this presents the issue that a different sized window will need to be used depending on the RPM. An accurate measurement from 100 to 10,000 RPM is required.

I'm now trying a method where we turn the analog data into a digital signal where '1' is a reflector hit and a '0' is not. We can then get the timestamps of all the 1's and try to extract freq data out of that. This presents the problem of having multiple 1's per revolution.